In [1]:
pwd

'/content'

In [2]:
#Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Load the dataset
df=pd.read_csv("drive/My Drive/CSV_Files/DrugsDotCom_preprocessed.csv",encoding="ISO-8859-1")
df.head()

,Unnamed: 0,index,drugName,condition,review,rating,date,usefulCount,cleaned_review,tokenized,lemmatized
0,0,0,Liraglutide,Obesity,""" began with the minimum 0.6 mg dose of Saxend...",10.0,"March 18, 2017",28.0,[ began minimum mg dose saxenda february incre...,"['[', 'began', 'minimum', 'mg', 'dose', 'saxen...",[ began minimum mg dose saxenda february incre...
1,1,1,Quetiapine,Bipolar Disorde,""" Caused depression and negative, self defeati...",1.0,"August 25, 2016",2.0,[ caused depression negative self defeating th...,"['[', 'caused', 'depression', 'negative', 'sel...",[ caused depression negative self defeating th...
2,2,2,Ethinyl estradiol / levonorgestrel,Birth Control,""" Ever since I started taking this birth contr...",1.0,"October 30, 2017",2.0,[ ever since started taking birth control noti...,"['[', 'ever', 'since', 'started', 'taking', 'b...",[ ever since started taking birth control noti...
3,3,3,Lorazepam,Anxiety,""" Had a sort of nervous breakdown during a tri...",9.0,"June 4, 2014",64.0,[ sort nervous breakdown trip could catch brea...,"['[', 'sort', 'nervous', 'breakdown', 'trip', ...",[ sort nervous breakdown trip could catch brea...
4,4,4,Aczone,Acne,""" I am 28, have struggled with acne for 10 yea...",10.0,"June 2, 2015",16.0,[ struggled acne year product kept consistentl...,"['[', 'struggled', 'acne', 'year', 'product', ...",[ struggled acne year product kept consistentl...


In [5]:
import nltk
from nltk.corpus import wordnet
import string
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [6]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [7]:
from textblob import TextBlob

In [8]:
df['polarity'] = df['lemmatized'].apply(lambda x: TextBlob(x).sentiment.polarity)
df.head()

,Unnamed: 0,index,drugName,condition,review,rating,date,usefulCount,cleaned_review,tokenized,lemmatized,polarity
0,0,0,Liraglutide,Obesity,""" began with the minimum 0.6 mg dose of Saxend...",10.0,"March 18, 2017",28.0,[ began minimum mg dose saxenda february incre...,"['[', 'began', 'minimum', 'mg', 'dose', 'saxen...",[ began minimum mg dose saxenda february incre...,-0.128571
1,1,1,Quetiapine,Bipolar Disorde,""" Caused depression and negative, self defeati...",1.0,"August 25, 2016",2.0,[ caused depression negative self defeating th...,"['[', 'caused', 'depression', 'negative', 'sel...",[ caused depression negative self defeating th...,-0.299621
2,2,2,Ethinyl estradiol / levonorgestrel,Birth Control,""" Ever since I started taking this birth contr...",1.0,"October 30, 2017",2.0,[ ever since started taking birth control noti...,"['[', 'ever', 'since', 'started', 'taking', 'b...",[ ever since started taking birth control noti...,0.193750
3,3,3,Lorazepam,Anxiety,""" Had a sort of nervous breakdown during a tri...",9.0,"June 4, 2014",64.0,[ sort nervous breakdown trip could catch brea...,"['[', 'sort', 'nervous', 'breakdown', 'trip', ...",[ sort nervous breakdown trip could catch brea...,0.092929
4,4,4,Aczone,Acne,""" I am 28, have struggled with acne for 10 yea...",10.0,"June 2, 2015",16.0,[ struggled acne year product kept consistentl...,"['[', 'struggled', 'acne', 'year', 'product', ...",[ struggled acne year product kept consistentl...,0.003939


In [9]:
df['polarity1'] = df['lemmatized'].apply(lambda x: SentimentIntensityAnalyzer().polarity_scores(x))
df.head(2)

,Unnamed: 0,index,drugName,condition,review,rating,date,usefulCount,cleaned_review,tokenized,lemmatized,polarity,polarity1
0,0,0,Liraglutide,Obesity,""" began with the minimum 0.6 mg dose of Saxend...",10.0,"March 18, 2017",28.0,[ began minimum mg dose saxenda february incre...,"['[', 'began', 'minimum', 'mg', 'dose', 'saxen...",[ began minimum mg dose saxenda february incre...,-0.128571,"{'neg': 0.296, 'neu': 0.596, 'pos': 0.109, 'co..."
1,1,1,Quetiapine,Bipolar Disorde,""" Caused depression and negative, self defeati...",1.0,"August 25, 2016",2.0,[ caused depression negative self defeating th...,"['[', 'caused', 'depression', 'negative', 'sel...",[ caused depression negative self defeating th...,-0.299621,"{'neg': 0.583, 'neu': 0.319, 'pos': 0.099, 'co..."


In [10]:
df.drop(['review','date','usefulCount','cleaned_review','tokenized'],axis=1)

,Unnamed: 0,index,drugName,condition,rating,lemmatized,polarity,polarity1
0,0,0,Liraglutide,Obesity,10.0,[ began minimum mg dose saxenda february incre...,-0.128571,"{'neg': 0.296, 'neu': 0.596, 'pos': 0.109, 'co..."
1,1,1,Quetiapine,Bipolar Disorde,1.0,[ caused depression negative self defeating th...,-0.299621,"{'neg': 0.583, 'neu': 0.319, 'pos': 0.099, 'co..."
2,2,2,Ethinyl estradiol / levonorgestrel,Birth Control,1.0,[ ever since started taking birth control noti...,0.193750,"{'neg': 0.196, 'neu': 0.734, 'pos': 0.07, 'com..."
3,3,3,Lorazepam,Anxiety,9.0,[ sort nervous breakdown trip could catch brea...,0.092929,"{'neg': 0.125, 'neu': 0.581, 'pos': 0.294, 'co..."
4,4,4,Aczone,Acne,10.0,[ struggled acne year product kept consistentl...,0.003939,"{'neg': 0.091, 'neu': 0.613, 'pos': 0.295, 'co..."
...,...,...,...,...,...,...,...,...
113496,113496,128744,Zyrtec,Allergic Rhinitis,2.0,[ zyrtec worked well nasal allergy tried stop ...,0.118750,"{'neg': 0.271, 'neu': 0.594, 'pos': 0.135, 'co..."
113497,113497,128745,Zyrtec,Allergic Rhinitis,3.0,[ zyrtec worked well control seasonal allergy ...,-0.146429,"{'neg': 0.154, 'neu': 0.696, 'pos': 0.15, 'com..."
113498,113498,128746,Zyrtec,Allergic Rhinitis,5.0,[ zyrtec work incredibly well severe ragweed a...,0.333333,"{'neg': 0.168, 'neu': 0.64, 'pos': 0.192, 'com..."
113499,113499,128747,Zyrtec,Allergic Rhinitis,10.0,[ zyrtec work well combating allergy well alle...,0.637500,"{'neg': 0.053, 'neu': 0.483, 'pos': 0.464, 'co..."


In [11]:
df['polarity2'] = df['lemmatized'].apply(lambda x: SentimentIntensityAnalyzer().polarity_scores(x)['compound'])
df.head(2)

,Unnamed: 0,index,drugName,condition,review,rating,date,usefulCount,cleaned_review,tokenized,lemmatized,polarity,polarity1,polarity2
0,0,0,Liraglutide,Obesity,""" began with the minimum 0.6 mg dose of Saxend...",10.0,"March 18, 2017",28.0,[ began minimum mg dose saxenda february incre...,"['[', 'began', 'minimum', 'mg', 'dose', 'saxen...",[ began minimum mg dose saxenda february incre...,-0.128571,"{'neg': 0.296, 'neu': 0.596, 'pos': 0.109, 'co...",-0.9274
1,1,1,Quetiapine,Bipolar Disorde,""" Caused depression and negative, self defeati...",1.0,"August 25, 2016",2.0,[ caused depression negative self defeating th...,"['[', 'caused', 'depression', 'negative', 'sel...",[ caused depression negative self defeating th...,-0.299621,"{'neg': 0.583, 'neu': 0.319, 'pos': 0.099, 'co...",-0.9866


In [ ]:
df1=df.groupby('drugName')

In [ ]:
PosEffect=df1.get_group('Liraglutide')

In [ ]:
filter=PosEffect['rating']>9
filter1=PosEffect['polarity']>0
PosEffect=PosEffect.where(filter&filter1==True)
PosEffect=PosEffect.dropna()

In [ ]:
PosEffect


,Unnamed: 0,index,drugName,condition,review,rating,date,usefulCount,cleaned_review,tokenized,lemmatized,polarity,polarity1,polarity2
118,118.0,146.0,Liraglutide,Obesity,"""**4th UPDATE** UPDATE* I have now been on Sax...",10.0,"July 24, 2017",45.0,[ th update update saxenda since feb th lb jul...,"['[', 'th', 'update', 'update', 'saxenda', 'si...",[ th update update saxenda since feb th lb jul...,0.118333,"{'neg': 0.064, 'neu': 0.657, 'pos': 0.279, 'co...",0.9460
120,120.0,150.0,Liraglutide,Obesity,"""**Update** I have now been on Saxenda for 10 ...",10.0,"May 7, 2016",36.0,[ update saxenda week lost pound eat calorie d...,"['[', 'update', 'saxenda', 'week', 'lost', 'po...",[ update saxenda week lost pound eat calorie d...,0.600000,"{'neg': 0.218, 'neu': 0.647, 'pos': 0.135, 'co...",-0.4226
121,121.0,151.0,Liraglutide,Obesity,"""**UPDATE** I have now been on Saxenda since 2...",10.0,"April 4, 2017",18.0,[ update saxenda since april st day saxenda we...,"['[', 'update', 'saxenda', 'since', 'april', '...",[ update saxenda since april st day saxenda we...,0.034745,"{'neg': 0.149, 'neu': 0.785, 'pos': 0.066, 'co...",-0.7190
122,122.0,153.0,Liraglutide,Obesity,"""*3rd update* I started Saxenda 2-19-2017 star...",10.0,"April 24, 2017",42.0,[ rd update started saxenda starting weight da...,"['[', 'rd', 'update', 'started', 'saxenda', 's...",[ rd update started saxenda starting weight da...,0.200000,"{'neg': 0.123, 'neu': 0.685, 'pos': 0.192, 'co...",0.4588
126,126.0,163.0,Liraglutide,Obesity,"""*Update #3 or 4 lol.. I&#039;ve been on Saxen...",10.0,"May 17, 2017",33.0,[ update lol saxenda current dose lost total l...,"['[', 'update', 'lol', 'saxenda', 'current', '...",[ update lol saxenda current dose lost total l...,0.246561,"{'neg': 0.145, 'neu': 0.565, 'pos': 0.29, 'com...",0.9001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108366,108366.0,123003.0,Liraglutide,"Diabetes, Type 2","""Victoza is an amazing drug thank you. My insu...",10.0,"August 23, 2017",19.0,[ victoza amazing drug thank insurance covered...,"['[', 'victoza', 'amazing', 'drug', 'thank', '...",[ victoza amazing drug thank insurance covered...,0.080000,"{'neg': 0.085, 'neu': 0.632, 'pos': 0.283, 'co...",0.9509
108967,108967.0,123690.0,Liraglutide,"Diabetes, Type 2","""Was on victoza 4 years ago and lost about 48 ...",10.0,"March 29, 2016",61.0,[ victoza year ago lost pound c switched insur...,"['[', 'victoza', 'year', 'ago', 'lost', 'pound...",[ victoza year ago lost pound c switched insur...,0.127851,"{'neg': 0.062, 'neu': 0.763, 'pos': 0.175, 'co...",0.7906
110003,110003.0,124872.0,Liraglutide,"Diabetes, Type 2","""Went off of Byetta in July, 2009 -- never fel...",10.0,"April 4, 2010",42.0,[ went byetta july never felt quite well incon...,"['[', 'went', 'byetta', 'july', 'never', 'felt...",[ went byetta july never felt quite well incon...,0.245536,"{'neg': 0.109, 'neu': 0.732, 'pos': 0.159, 'co...",0.5091
110363,110363.0,125275.0,Liraglutide,"Diabetes, Type 2","""When I first started Victoria I got very sick...",10.0,"August 21, 2013",44.0,[ first started victoria got sick waited week ...,"['[', 'first', 'started', 'victoria', 'got', '...",[ first started victoria got sick waited week ...,0.287798,"{'neg': 0.09, 'neu': 0.592, 'pos': 0.318, 'com...",0.9231


In [ ]:
# count vectoriser tells the frequency of a word.
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#Single Word
def get_top_n1_words(corpus, n=None):
    vec1 = CountVectorizer(ngram_range=(1,1)).fit(corpus)
    bag_of_words = vec1.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     
                  vec1.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                reverse=True)
    return words_freq[:n]

In [ ]:
top_words = get_top_n1_words(PosEffect["lemmatized"], n=10) #top 5
top_df = pd.DataFrame(top_words)
top_df.columns=["Word", "Freq"]
top_df.head(10)

,Word,Freq
0,lb,221
1,day,206
2,week,205
3,lost,178
4,weight,178
5,started,172
6,victoza,167
7,side,159
8,effect,154
9,pound,117


In [ ]:
#Bi-gram
def get_top_n2_words(corpus, n=None):
    vec1 = CountVectorizer(ngram_range=(2,2)).fit(corpus)
    bag_of_words = vec1.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     
                  vec1.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                reverse=True)
    return words_freq[:n]

In [ ]:
top2_words = get_top_n2_words(PosEffect["lemmatized"], n=10) #top 10
top2_df = pd.DataFrame(top2_words)
top2_df.columns=["Bi-gram", "Freq"]
top2_df.head(10)

,Bi-gram,Freq
0,side effect,150
1,lost lb,74
2,weight loss,54
3,lost pound,51
4,blood sugar,44
5,started saxenda,29
6,week lost,25
7,first day,24
8,first week,21
9,week ago,19


In [ ]:
#Tri-gram
def get_top_n3_words(corpus, n=None):
    vec1 = CountVectorizer(ngram_range=(3,3)).fit(corpus)
    bag_of_words = vec1.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     
                  vec1.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                reverse=True)
    return words_freq[:n]


In [ ]:
top3_words = get_top_n3_words(PosEffect["lemmatized"], n=5)
top3_df = pd.DataFrame(top3_words)
top3_df.columns=["Tri-gram", "Freq"]
top3_df.head(5)

,Tri-gram,Freq
0,week lost lb,13
1,day lost lb,7
2,blood sugar level,7
3,low carb diet,7
4,lb side effect,6


In [ ]:
NeEffectt=df1.get_group('Liraglutide')
filter2=NeEffectt['rating']<5
filter3=NeEffectt['polarity']< 0
NeEffectt=NeEffectt.where(filter2&filter3==True)
NeEffectt=NeEffectt.dropna()

In [ ]:
NeEffectt

,Unnamed: 0,index,drugName,condition,review,rating,date,usefulCount,cleaned_review,tokenized,lemmatized,polarity,polarity1,polarity2
5226,5226.0,6147.0,Liraglutide,"Diabetes, Type 2","""At first the side effects were horrible, naus...",2.0,"March 24, 2016",36.0,[ first side effect horrible nausea diarrhea e...,"['[', 'first', 'side', 'effect', 'horrible', '...",[ first side effect horrible nausea diarrhea e...,-0.243750,"{'neg': 0.419, 'neu': 0.581, 'pos': 0.0, 'comp...",-0.9792
6152,6152.0,7190.0,Liraglutide,Weight Loss,"""Been on it over a month and have gained weigh...",1.0,"December 6, 2016",8.0,[ month gained weight lost sad bc side effect ...,"['[', 'month', 'gained', 'weight', 'lost', 'sa...",[ month gained weight lost sad bc side effect ...,-0.500000,"{'neg': 0.36, 'neu': 0.467, 'pos': 0.173, 'com...",-0.4215
6475,6475.0,7549.0,Liraglutide,"Diabetes, Type 2","""Been on victoza 0.6 mg for a week. Was on a f...",1.0,"February 5, 2017",5.0,[ victoza mg week flozin wonderful lost stone ...,"['[', 'victoza', 'mg', 'week', 'flozin', 'wond...",[ victoza mg week flozin wonderful lost stone ...,-0.040476,"{'neg': 0.261, 'neu': 0.56, 'pos': 0.179, 'com...",-0.8979
9730,9730.0,11251.0,Liraglutide,"Diabetes, Type 2","""Didn&#039;t do bad on .06 the first week but ...",1.0,"July 23, 2015",43.0,[ bad first week still strong urge eat diarrhe...,"['[', 'bad', 'first', 'week', 'still', 'strong...",[ bad first week still strong urge eat diarrhe...,-0.040741,"{'neg': 0.134, 'neu': 0.77, 'pos': 0.096, 'com...",-0.4588
10725,10725.0,12399.0,Liraglutide,Weight Loss,"""Due to my genetics and hormone imbalances it ...",3.0,"June 30, 2016",4.0,[ due genetics hormone imbalance really hard l...,"['[', 'due', 'genetics', 'hormone', 'imbalance...",[ due genetics hormone imbalance really hard l...,-0.140196,"{'neg': 0.301, 'neu': 0.527, 'pos': 0.172, 'co...",-0.9143
17613,17613.0,20295.0,Liraglutide,Obesity,"""Hi There I&#039;ve been on Saxenda for three ...",1.0,"October 17, 2016",15.0,[ hi saxenda three week result actually put we...,"['[', 'hi', 'saxenda', 'three', 'week', 'resul...",[ hi saxenda three week result actually put we...,-0.204545,"{'neg': 0.147, 'neu': 0.853, 'pos': 0.0, 'comp...",-0.4767
18342,18342.0,21153.0,Liraglutide,"Diabetes, Type 2","""I absolutely do NOT recommend this medication...",1.0,"February 8, 2017",14.0,[ absolutely recommend medication tried help w...,"['[', 'absolutely', 'recommend', 'medication',...",[ absolutely recommend medication tried help w...,-0.190000,"{'neg': 0.367, 'neu': 0.526, 'pos': 0.106, 'co...",-0.9729
22003,22003.0,25282.0,Liraglutide,Obesity,"""I am on my third box of Saxenda and I have no...",1.0,"November 16, 2015",18.0,[ third box saxenda lost one ounce lost pound ...,"['[', 'third', 'box', 'saxenda', 'lost', 'one'...",[ third box saxenda lost one ounce lost pound ...,-0.020000,"{'neg': 0.153, 'neu': 0.706, 'pos': 0.141, 'co...",-0.4215
23108,23108.0,26528.0,Liraglutide,"Diabetes, Type 2","""I began taking this drug at the 1.2 mg level....",1.0,"May 4, 2016",27.0,[ began taking drug mg level within half day b...,"['[', 'began', 'taking', 'drug', 'mg', 'level'...",[ began taking drug mg level within half day b...,-0.221667,"{'neg': 0.239, 'neu': 0.698, 'pos': 0.063, 'co...",-0.9022
23163,23163.0,26587.0,Liraglutide,Obesity,"""I began this medication 6 weeks ago I read re...",2.0,"October 10, 2016",23.0,[ began medication week ago read review first ...,"['[', 'began', 'medication', 'week', 'ago', 'r...",[ began medication week ago read review first ...,-0.070536,"{'neg': 0.19, 'neu': 0.634, 'pos': 0.175, 'com...",-0.4767


In [ ]:
top_words = get_top_n1_words(NeEffectt["lemmatized"], n=10) #top 10
top_df = pd.DataFrame(top_words)
top_df.columns=["Word", "Freq"]
top_df.head(10)

,Word,Freq
0,day,36
1,weight,36
2,week,32
3,lost,31
4,victoza,27
5,month,26
6,started,25
7,side,24
8,effect,23
9,saxenda,23


In [ ]:
top2_words = get_top_n2_words(NeEffectt["lemmatized"], n=10) #top 10
top2_df = pd.DataFrame(top2_words)
top2_df.columns=["Bi-gram", "Freq"]
top2_df.head(10)

,Bi-gram,Freq
0,side effect,22
1,weight loss,17
2,lost lb,10
3,blood sugar,10
4,first week,6
5,month lost,6
6,lost weight,6
7,stomach pain,5
8,nausea vomiting,5
9,feel like,5


In [ ]:
top3_words = get_top_n3_words(NeEffectt["lemmatized"], n=5)
top3_df = pd.DataFrame(top3_words)
top3_df.columns=["Tri-gram", "Freq"]
top3_df.head(5)

,Tri-gram,Freq
0,weight loss yet,3
1,awful side effect,2
2,weight side effect,2
3,side effect disappointed,2
4,nausea headache first,2


In [ ]:
#Using Plaority2 col
filter=PosEffect['rating']>9
filter1=PosEffect['polarity2']>0.5
PosEffect=PosEffect.where(filter&filter1==True)
PosEffect=PosEffect.dropna()

In [ ]:
top_words = get_top_n1_words(PosEffect["lemmatized"], n=10) #top 5
top_df = pd.DataFrame(top_words)
top_df.columns=["Word", "Freq"]
top_df.head(10)

,Word,Freq
0,lb,130
1,day,119
2,week,119
3,started,103
4,victoza,102
5,side,93
6,effect,90
7,lost,86
8,weight,84
9,feel,84


In [ ]:
top2_words = get_top_n2_words(PosEffect["lemmatized"], n=10) #top 10
top2_df = pd.DataFrame(top2_words)
top2_df.columns=["Bi-gram", "Freq"]
top2_df.head(10)

,Bi-gram,Freq
0,side effect,88
1,lost lb,37
2,blood sugar,35
3,lost pound,22
4,weight loss,19
5,started saxenda,16
6,feel great,15
7,sugar level,14
8,first week,12
9,first day,11


In [ ]:
top3_words = get_top_n3_words(PosEffect["lemmatized"], n=5)
top3_df = pd.DataFrame(top3_words)
top3_df.columns=["Tri-gram", "Freq"]
top3_df.head(5)

,Tri-gram,Freq
0,blood sugar level,6
1,major side effect,4
2,doctor put victoza,4
3,day lost lb,4
4,side effect nausea,4


In [ ]:
filter2=NeEffectt['rating']<5
filter3=NeEffectt['polarity']< -0.5
NeEffectt=NeEffectt.where(filter2&filter3==True)
NeEffectt=NeEffectt.dropna()

In [ ]:
top_words = get_top_n1_words(NeEffectt["lemmatized"], n=10) #top 5
top_df = pd.DataFrame(top_words)
top_df.columns=["Word", "Freq"]
top_df.head(10)

,Word,Freq
0,weight,5
1,horrible,3
2,lost,3
3,started,3
4,mg,3
5,ago,3
6,time,3
7,sick,3
8,side,2
9,effect,2


In [ ]:
top2_words = get_top_n2_words(NeEffectt["lemmatized"], n=10) #top 10
top2_df = pd.DataFrame(top2_words)
top2_df.columns=["Bi-gram", "Freq"]
top2_df.head(10)

,Bi-gram,Freq
0,lost weight,3
1,side effect,2
2,weight loss,2
3,week ago,2
4,horrible side,1
5,effect including,1
6,including severe,1
7,severe indigestion,1
8,indigestion diarrhea,1
9,diarrhea vomiting,1


In [ ]:
top3_words = get_top_n3_words(NeEffectt["lemmatized"], n=5)
top3_df = pd.DataFrame(top3_words)
top3_df.columns=["Tri-gram", "Freq"]
top3_df.head(5)

,Tri-gram,Freq
0,horrible side effect,1
1,side effect including,1
2,effect including severe,1
3,including severe indigestion,1
4,severe indigestion diarrhea,1
